In [ ]:
#r "nuget: Jinaga.UnitTest, 0.1.1"

Installed Packages Jinaga.UnitTest, 0.1.1

In [ ]:
using Jinaga.UnitTest;
using Jinaga;

[FactType("Model.Company")]
record Company(string identifier);

[FactType("Jinaga.User")]
record User(string publicKey);

[FactType("Model.Employment")]
record Employment(Company company, User user, DateTime hireDate);

[FactType("Model.Termination")]
record Termination(Employment employment);

I've developed four specification languages for expressing interest. Each was designed for a different context. Here is the same model in each language.

First, in the Factual modeling language as seen in the book:

```factual
query allEmployeesOfCompany(c: Company) {
  match e: Employment where e.company = c
    such that not exists t: Termination where t.employment = e
}
```

Second, as implemented in Jinaga.JS:

```js
function allEmployeesOfCompany(company) {
  return j.match({
    type: Employment.Type
    company
  }).suchThat(j.not(terminationOfEmployement));
}

function terminationOfEmployment(employment) {
  return j.exist({
    type: Termination.Type
    employment
  });
}
```

Third, in Jinaga.NET:

In [ ]:
var allEmployeesOfCompany = Given<Company>.Match((company, facts) =>
  from employment in facts.OfType<Employment>()
  where employment.company == company
  where !(
    from termination in facts.OfType<Termination>()
    where termination.employment == employment
    select termination
  ).Any()
  select employment
);

And finally, in the wire specification generated by Jinaga.NET (and in the near future Jinaga.JS):

In [ ]:
allEmployeesOfCompany.Pipeline.ToDescriptiveString() + allEmployeesOfCompany.Projection.ToDescriptiveString()

company: Model.Company {
    employment: Model.Employment = company S.company Model.Employment
    N(
        employment: Model.Employment {
            termination: Model.Termination = employment S.employment Model.Termination
        }
    )
}
employment